In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import numpy as np 
import pandas as pd 
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/X_copy.pkl
/kaggle/input/dataset/y_copy.pkl


In [11]:
import pickle

with open('/kaggle/input/dataset/X_copy.pkl', 'rb') as f:
    X_copy = pickle.load(f)

with open('/kaggle/input/dataset/y_copy.pkl', 'rb') as f:
    y_copy = pickle.load(f)

In [12]:
X_copy

,WP,Bore,Stroke,Clearance,Notch Angle degree,diff,Clearance_area
0,-0.03,0.90,0.29,-1.21,0.15,1.19,4.64
1,-0.03,-0.51,0.27,0.11,0.45,1.13,0.11
2,-0.03,0.90,0.29,0.10,0.45,1.10,0.52
3,-0.03,0.90,0.29,-1.50,0.08,1.19,7.15
4,-0.03,0.90,0.29,-0.12,-1.48,2.83,-0.62
...,...,...,...,...,...,...,...
78,-0.03,0.90,0.29,-0.12,0.45,0.03,-0.62
79,-0.03,-0.51,0.27,-0.08,0.45,0.02,-0.03
80,1.46,-1.21,-1.80,1.12,0.95,7.26,-14.89
81,-0.03,0.90,0.29,0.10,0.45,0.03,0.60


In [79]:
X_copy

,WP,Bore,Stroke,Clearance,Notch Angle degree,diff,Clearance_area
0,-0.03,0.90,0.29,-1.21,0.15,-0.18,0.16
1,-0.03,-0.51,0.27,0.11,0.45,-0.19,-0.05
2,-0.03,0.90,0.29,0.10,0.45,-0.20,-0.03
3,-0.03,0.90,0.29,-1.50,0.08,-0.18,0.27
4,-0.03,0.90,0.29,-0.12,-1.48,0.23,-0.08
...,...,...,...,...,...,...,...
78,-0.03,0.90,0.29,-0.12,0.45,-0.46,-0.08
79,-0.03,-0.51,0.27,-0.08,0.45,-0.47,-0.05
80,1.46,-1.21,-1.80,1.12,0.95,1.31,-0.72
81,-0.03,0.90,0.29,0.10,0.45,-0.46,-0.03


In [20]:
ex = pd.DataFrame(np.array(X_copy.iloc[76]).reshape(1,-1), columns = X_copy.columns)

In [55]:
y_copy.iloc[76]

1

In [127]:
def make_df(user_input):
    Bush_od_min = 55
    Bush_od_max = 57
    clear_min = 0.04
    clear_max = 0.11
#     df['Notch length(mm)'].max()
# df['Notch Angle degree'].max()
    sample = user_input.copy()
    sample['Bush OD'] = round(np.random.uniform(Bush_od_min, Bush_od_max), 3)
    sample['Clearance'] = round(np.random.uniform(clear_min, clear_max), 3)
    sample['Notches'] = np.random.randint(1, 5)
    sample['Notch Angle degree'] = round(np.random.uniform(0, 0.5), 3)
    sample['Notch length(mm)'] = round(np.random.uniform(5, user_input['Bush Length']), 3)
    sample_df = pd.DataFrame([sample])

    cols_to_drop = ['Type', 'Tons' ,'Bush Length'] 
    sample_df_test = sample_df.drop(cols_to_drop , axis= 1)

    scaler = StandardScaler()
    sample_df_test = pd.DataFrame(scaler.fit_transform(sample_df_test),columns = sample_df_test.columns)
    sample_df_test['diff'] = ((sample_df_test['Notches']-sample_df_test['Notch length(mm)']))**2 
    sample_df_test['Clearance_area'] = np.pi*(2*sample_df_test['Bush OD']*sample_df_test['Clearance'] + (sample_df_test['Clearance'])**2)
    sample_df_test = sample_df_test.drop(['Notches','Notch length(mm)','Rod', 'Bush OD'] , axis= 1)
    
#     sample_df['diff'] = ((sample_df['Notches'] - sample_df['Notch length(mm)']))**2 
#     sample_df['Clearance_area'] = np.pi * (2 * sample_df['Bush OD'] * sample_df['Clearance'] + (sample_df['Clearance'])**2)
#     sample_df_test = sample_df.drop(['Notches', 'Notch length(mm)', 'Rod', 'Bush OD'], axis=1)
    
    return sample_df,sample_df_test

In [128]:
def make_predictions(X_copy ,y_copy,ex , scaler):
    accuracy_scores = []
    classification_reports = []
    predictions = []
    dim_predictions = []
    prediction_probabilities = []

    ex = scaler.transform(ex)
    
    for i in range(16):
        X_train, X_test, y_train, y_test = train_test_split(X_copy.iloc[:69], y_copy.iloc[:69], test_size=0.2, random_state=i)

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        xgb_classifier = XGBClassifier(random_state=4, max_depth=3, learning_rate=0.214, objective='binary:logistic')

        xgb_classifier.fit(X_train, y_train)

        y_pred = xgb_classifier.predict(X_test)
        y_new = xgb_classifier.predict(ex)
        y_prob = xgb_classifier.predict_proba(ex)[:, 1]
        
        predictions.append(y_new[0])
        prediction_probabilities.append(y_prob[0])
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        accuracy_scores.append(accuracy)
        classification_reports.append(report)

    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)

    mean_prediction = np.mean(predictions)
    mean_probability = np.mean(prediction_probabilities)
    print(f"Mean prediction for the new example: {mean_prediction}")
    ans = 1 if mean_prediction > 0.5 else 0
    if(ans == 1):
        dim_predictions.append(ex)
        print(scaler.inverse_transform(ex))
    print(f"Bush fine? {ans}")
    return dim_predictions, mean_probability

In [129]:
Bush_od_min = 55
Bush_od_max = 55.5
clear_min = 0.04
clear_max = 0.11

search_space = [
    Real(55, 56, name='Bush OD'),
    Real(0.04, 0.11, name='Clearance'),
    Integer(0, 4, name='Notches'),
    Real(0, 0.5, name='Notch Angle degree'),
    Real(5, 35, name='Notch length(mm)')  # Assuming 'Bush Length' is 35
]

scaler = StandardScaler()
X_copy = pd.DataFrame(scaler.fit_transform(X_copy),columns = X_copy.columns)

@use_named_args(search_space)
def objective(**params):
    sample = {
        'WP': 240,  # Fixed for this example
        'Bore ': 80,  # Fixed for this example
        'Stroke': 698.5,  # Fixed for this example
        'Clearance': params['Clearance'],
        'Bush OD': params['Bush OD'],
        'Notch Angle degree': params['Notch Angle degree'],
        'Notches': params['Notches'],
        'Notch length(mm)': params['Notch length(mm)'],
        'Rod': 50, 
    }
    sample_df = pd.DataFrame([sample])

    sample_df['diff'] = (sample_df['Notches'] - sample_df['Notch length(mm)'])**2
    sample_df['Clearance_area'] = np.pi * (2 * sample_df['Bush OD'] * sample_df['Clearance'] + (sample_df['Clearance'])**2)
    
    sample_df_test = sample_df.drop(['Notches', 'Notch length(mm)', 'Rod', 'Bush OD'], axis=1)

    dim_predictions, mean_probability = make_predictions(X_copy , y_copy , sample_df_test , scaler)
    
    return -mean_probability # Minimize the negative of the mean probability for optimization


In [131]:
res = gp_minimize(objective, search_space, n_calls=100, random_state=0)

# Best result
print("Best parameters found:")
print(res.x)
print("Best objective value:")
print(-res.fun)

Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.625
[[2.40000000e+02 8.00000000e+01 6.98500000e+02 5.55624972e-02
  4.51299238e-01 2.72200492e+02 1.92629722e+01]]
Bush fine? 1
Mean prediction for the new example: 0.625
[[2.40000000e+02 8.00000000e+01 6.98500000e+02 1.03164401e-01
  4.84904534e-01 6.04874726e+02 3.60818816e+01]]
Bush fine? 1
Mean prediction for the new example: 0.5
Bush fine? 0
Mean prediction for the new example: 0.625
[[2.40000000e+02 8.00000000e+01 6.98500000e+02 4.00000000e-02
  4.65360410e-01 1.36434339e+02 1.38311039e+01]]
Bush fine? 1
Mean prediction for the new example: 0.625
[[2.40000000e+02 8.00000

In [132]:
X_copy.columns
search_space = [
    Real(55, 56, name='Bush OD'),
    Real(0.04, 0.11, name='Clearance'),
    Integer(0, 4, name='Notches'),
    Real(0, 0.5, name='Notch Angle degree'),
    Real(5, 35, name='Notch length(mm)')  # Assuming 'Bush Length' is 35
]

Index(['WP', 'Bore ', 'Stroke', 'Clearance', 'Notch Angle degree', 'diff',
       'Clearance_area'],
      dtype='object')

In [60]:
user_input = {'Tons': 4.8, 'Bush Length': 35, 'WP': 240, 'Bore ': 80, 'Rod': 50, 'Stroke': 698.5, 'Type': 1 }  # Example user input
samples = 100
possible_dims = []
for i in range(samples):
    print(f'progress: {i}')
    sample_df,sample_df_test = make_df(user_input)
    dim_predictions = make_predictions(sample_df_test)
    if(len(dim_predictions) != 0):
        possible_dims.append(dim_predictions)


progress: 0
Bush fine? 0
progress: 1
Bush fine? 0
progress: 2
Bush fine? 0
progress: 3
Bush fine? 0
progress: 4
Bush fine? 0
progress: 5
Bush fine? 0
progress: 6
Bush fine? 0
progress: 7
Bush fine? 0
progress: 8
Bush fine? 0
progress: 9
Bush fine? 0
progress: 10
Bush fine? 0
progress: 11
Bush fine? 0
progress: 12
Bush fine? 0
progress: 13
Bush fine? 0
progress: 14
Bush fine? 0
progress: 15
Bush fine? 0
progress: 16
Bush fine? 0
progress: 17
Bush fine? 0
progress: 18
Bush fine? 0
progress: 19
Bush fine? 0
progress: 20
Bush fine? 0
progress: 21
Bush fine? 0
progress: 22
Bush fine? 0
progress: 23
Bush fine? 0
progress: 24
Bush fine? 0
progress: 25
Bush fine? 0
progress: 26
Bush fine? 0
progress: 27
Bush fine? 0
progress: 28
Bush fine? 0
progress: 29
Bush fine? 0
progress: 30
Bush fine? 0
progress: 31
Bush fine? 0
progress: 32
Bush fine? 0
progress: 33
Bush fine? 0
progress: 34
Bush fine? 0
progress: 35
Bush fine? 0
progress: 36
Bush fine? 0
progress: 37
Bush fine? 0
progress: 38
Bush fine

In [61]:
possible_dims

[]